#06_SparkDataAnal.ipynb
TLC Trip Record Data
출처: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [ ]:
#2015-summery.json

In [2]:
df = spark.read.format('json')\
    .load("learning_spark_data/2015-summary.json")

In [3]:
df.count()

256

In [4]:
df.dtypes

[('DEST_COUNTRY_NAME', 'string'),
 ('ORIGIN_COUNTRY_NAME', 'string'),
 ('count', 'bigint')]

In [5]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [6]:
df.collect()

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Sint Maarten', count=325),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Marshall Islands', count=39),
 

In [7]:
df.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [8]:
df.select('count').show(5)

+-----+
|count|
+-----+
|   15|
|    1|
|  344|
|   15|
|   62|
+-----+
only showing top 5 rows



In [15]:
df.select('DEST_COUNTRY_NAME').distinct().show()

+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|            Anguilla|
|              Russia|
|            Paraguay|
|             Senegal|
|              Sweden|
|            Kiribati|
|              Guyana|
|         Philippines|
|            Djibouti|
|            Malaysia|
|           Singapore|
|                Fiji|
|              Turkey|
|                Iraq|
|             Germany|
|              Jordan|
|               Palau|
|Turks and Caicos ...|
|              France|
|              Greece|
+--------------------+
only showing top 20 rows



In [18]:
df1 = df.select('DEST_COUNTRY_NAME').distinct().cache() #cache를 안하면 RDD 내부에서 새로 만들어야 하는 일이 생김
df1.count()

132

In [20]:
# ROW class 를 이용한 단일 레코드 생성

from pyspark.sql import Row
myRow = Row('hello', None, 1, False)
myRow

<Row('hello', None, 1, False)>

In [21]:
# 새로운 컬럼 추가하기
from pyspark.sql.functions import expr

df3 = df.withColumn('withinCountry', expr('ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME')) #expr sql 표현식을 받아 생성
df3

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, withinCountry: boolean]

In [22]:
df3.show(3)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
|    United States|            Ireland|  344|        false|
+-----------------+-------------------+-----+-------------+
only showing top 3 rows



In [27]:
df3.filter("withinCountry = TRUE").show()

+-----------------+-------------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|withinCountry|
+-----------------+-------------------+------+-------------+
|    United States|      United States|370002|         true|
+-----------------+-------------------+------+-------------+



In [28]:
df3.filter(expr('withinCountry==True')).show()

+-----------------+-------------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|withinCountry|
+-----------------+-------------------+------+-------------+
|    United States|      United States|370002|         true|
+-----------------+-------------------+------+-------------+



In [ ]:
# case when 카운트 10이하 under, 이상 upper 변환 > category 컬럼 추가

# df3.withColumn('category', expr('count<=0 == under', 'count>=0 == upper'))

In [32]:
df4 = df3.withColumn(
    'category',
    expr("CASE WHEN count <= 10 THEN 'under' ELSE 'upper' END")
)
df4.show(10)

+-----------------+-------------------+-----+-------------+--------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|category|
+-----------------+-------------------+-----+-------------+--------+
|    United States|            Romania|   15|        false|   upper|
|    United States|            Croatia|    1|        false|   under|
|    United States|            Ireland|  344|        false|   upper|
|            Egypt|      United States|   15|        false|   upper|
|    United States|              India|   62|        false|   upper|
|    United States|          Singapore|    1|        false|   under|
|    United States|            Grenada|   62|        false|   upper|
|       Costa Rica|      United States|  588|        false|   upper|
|          Senegal|      United States|   40|        false|   upper|
|          Moldova|      United States|    1|        false|   under|
+-----------------+-------------------+-----+-------------+--------+
only showing top 10 rows



In [ ]:
# DataFrame의 select(), where(), filter() 트렌스포메이션
# show(), count() 액션

In [33]:
spark.stop()

In [ ]:
# 집계함수

In [ ]:
# emp_df, dept_df

In [42]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyApp") \
    .getOrCreate()

In [43]:
emp_df = spark.read.format('csv')\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("/home/jovyan/work/learning_spark_data/emp.csv")

emp_df.show()


+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|NULL|    30|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|
| 9292|  JACK|

In [44]:
#filter()랑 동일
emp_df.select('*').where('deptno=20').show()

+-----+-----+-------+----+----------+----+----+------+
|empno|ename|    job| mgr|  hiredate| sal|comm|deptno|
+-----+-----+-------+----+----------+----+----+------+
| 7369|SMITH|  CLERK|7902|1980-12-17| 800|NULL|    20|
| 7566|JONES|MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7788|SCOTT|ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7876|ADAMS|  CLERK|7788|1987-05-23|1100|NULL|    20|
| 7902| FORD|ANALYST|7566|1981-12-03|3000|NULL|    20|
+-----+-----+-------+----+----------+----+----+------+



In [45]:
emp_df.selectExpr('count(*)').show()

+--------+
|count(1)|
+--------+
|      15|
+--------+



In [57]:
from pyspark.sql.functions import countDistinct

emp_df.select(countDistinct('job')).show()

+-------------------+
|count(DISTINCT job)|
+-------------------+
|                  5|
+-------------------+



In [58]:
from pyspark.sql.functions import approx_count_distinct
emp_df.select(approx_count_distinct('job', 0.1)).show()

+--------------------------+
|approx_count_distinct(job)|
+--------------------------+
|                         5|
+--------------------------+



In [ ]:
# first, last, min, max, sum, avg -> (expr : sql 문장) 아니고 function으로 처리하기
df.select( count)

In [59]:
from pyspark.sql.functions import first
emp_df.agg(first('ename', ignorenulls=True)).show()

+------------+
|first(ename)|
+------------+
|       SMITH|
+------------+



In [60]:
from pyspark.sql.functions import last
emp_df.agg(last('ename', ignorenulls=True)).show()

+-----------+
|last(ename)|
+-----------+
|       JACK|
+-----------+



In [67]:
from pyspark.sql.functions import min
emp_df.select(min('sal').alias('min_salary')).show()

+----------+
|min_salary|
+----------+
|       800|
+----------+



In [68]:
from pyspark.sql.functions import max
emp_df.select(max('sal').alias('max_salary')).show()

+----------+
|max_salary|
+----------+
|      5000|
+----------+



In [69]:
from pyspark.sql.functions import sum
emp_df.select(sum('sal').alias('sum_salary')).show()

+----------+
|sum_salary|
+----------+
|     32225|
+----------+



In [70]:
from pyspark.sql.functions import avg
emp_df.select(avg('sal').alias('avg_salary')).show()

+------------------+
|        avg_salary|
+------------------+
|2148.3333333333335|
+------------------+



In [72]:
from pyspark.sql.functions import mean
emp_df.select(mean('sal').alias('mean_salary')).show()

+------------------+
|       mean_salary|
+------------------+
|2148.3333333333335|
+------------------+



In [73]:
emp_df.select(sum('sal'), mean('sal'), avg('sal')).show()

+--------+------------------+------------------+
|sum(sal)|          avg(sal)|          avg(sal)|
+--------+------------------+------------------+
|   32225|2148.3333333333335|2148.3333333333335|
+--------+------------------+------------------+



In [74]:
from pyspark.sql.functions import col
emp_df.select(sum(col('sal'))).show()

+--------+
|sum(sal)|
+--------+
|   32225|
+--------+



In [75]:
emp_df.selectExpr('   sum(distinct sal)').show()

+-----------------+
|sum(DISTINCT sal)|
+-----------------+
|            27975|
+-----------------+



In [81]:
emp_df = emp_df.withColumn("total_salary", expr("sal + comm"))

emp_df.selectExpr(
    "sum(total_salary)", 
    "avg(total_salary)"
).show()

+-----------------+-----------------+
|sum(total_salary)|avg(total_salary)|
+-----------------+-----------------+
|             7800|           1950.0|
+-----------------+-----------------+



In [82]:
# total_salary / total_transaction, avg_salary, mean_salary

emp_df.selectExpr(
    "sum(sal) as total_salary",
    "count(sal) as total_transaction",
    "avg(sal) as avg_salary",
    "avg(sal) as mean_salary"
).show()

+------------+-----------------+------------------+------------------+
|total_salary|total_transaction|        avg_salary|       mean_salary|
+------------+-----------------+------------------+------------------+
|       32225|               15|2148.3333333333335|2148.3333333333335|
+------------+-----------------+------------------+------------------+



In [89]:
from pyspark.sql.functions import count, round

emp_df.select(
    sum('sal').alias ('total_salary'),
    count('sal').alias ('total_transaction'),
    avg('sal').alias ('avg_salary'),
    avg('sal').alias ('mean_salary')
).selectExpr(
    'total_salary/total_transaction',
    'avg_salary',
    'mean_salary'
).show()

+----------------------------------+------------------+------------------+
|(total_salary / total_transaction)|        avg_salary|       mean_salary|
+----------------------------------+------------------+------------------+
|                2148.3333333333335|2148.3333333333335|2148.3333333333335|
+----------------------------------+------------------+------------------+



In [ ]:
from pyspark.sql.functions import round

emp_df.select(
    sum('sal').alias ('total_salary'),
    count('sal').alias ('total_transaction'),
    avg('sal').alias ('avg_salary'),
    avg('sal').alias ('mean_salary')
).select(
    round((col('total_salary') / col('total_transaction')), 2).alias('avg_from_total'),
    round(col('avg_salary'), 2).alias('avg_salary'),
    round(col('mean_salary'), 2).alias('mean_salary')
).show()

In [90]:
#그룹화
emp_df.groupBy('job').count().show()

+---------+-----+
|      job|count|
+---------+-----+
|  ANALYST|    2|
| SALESMAN|    4|
|    CLERK|    5|
|  MANAGER|    3|
|PRESIDENT|    1|
+---------+-----+



In [92]:
# select job,
#     count(job),
#     sum(sal)
# group by job
group_df = emp_df.groupBy('job').agg(
    count('job').alias('qty'),
    expr('count(job)'),
    sum('sal')
)
group_df.show()

+---------+---+----------+--------+
|      job|qty|count(job)|sum(sal)|
+---------+---+----------+--------+
|  ANALYST|  2|         2|    6000|
| SALESMAN|  4|         4|    5600|
|    CLERK|  5|         5|    7350|
|  MANAGER|  3|         3|    8275|
|PRESIDENT|  1|         1|    5000|
+---------+---+----------+--------+



In [93]:

from pyspark.sql.functions import stddev
emp_df.groupBy("job") \
    .agg(stddev("sal").alias("salary_stddev")) \
    .show()


+---------+------------------+
|      job|     salary_stddev|
+---------+------------------+
|  ANALYST|               0.0|
| SALESMAN|177.95130420052183|
|    CLERK| 984.6319109189992|
|  MANAGER|274.24137786507225|
|PRESIDENT|              NULL|
+---------+------------------+



In [95]:
# sal의 평균 SAL_AVG, 표준편차 SAL_STDEV를 job별로 계산해서 출력, 소수점 2자리

from pyspark.sql.functions import avg, stddev, round

emp_df.agg(
    round(avg("sal"), 2).alias("SAL_AVG"),
    round(stddev("sal"), 2).alias("SAL_STDEV")
).show()


+-------+---------+
|SAL_AVG|SAL_STDEV|
+-------+---------+
|2148.33|  1176.04|
+-------+---------+



In [96]:
# 급여 TOP10 구하기 (feat.윈도우 함수)
from pyspark.sql.functions import desc, rank
from pyspark.sql.window import Window
windowspec = Window.orderBy(desc('sal'))
salAllRank = rank().over(windowspec)
salAllRank

Column<'RANK() OVER (ORDER BY sal DESC NULLS LAST unspecifiedframe$())'>

In [97]:
emp_df.show(10)

+-----+------+---------+----+----------+----+----+------+------------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|total_salary|
+-----+------+---------+----+----------+----+----+------+------------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|        NULL|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|        1900|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|        1750|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|        NULL|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|        2650|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|        NULL|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|        NULL|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|        NULL|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|        NULL|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|        1500|
+-----+------+---------+----+----------+----+----+------+------------+
only s

In [99]:
emp_df.withColumn('salary_rank', salAllRank).show(10)

+-----+------+---------+----+----------+----+----+------+------------+-----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|total_salary|salary_rank|
+-----+------+---------+----+----------+----+----+------+------------+-----------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|        NULL|          1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|        NULL|          2|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|        NULL|          3|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|        NULL|          3|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|        NULL|          5|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|        NULL|          6|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|        NULL|          7|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|        1900|          8|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|        1500|          9|
| 79

In [105]:
#job별로 rank 작성
#window.partitionBy()
job_window = Window.partitionBy("job").orderBy(col("sal").desc())
job_rank_df = emp_df.withColumn("salary_rank", rank().over(job_window))
job_rank_df.show()

+-----+------+---------+----+----------+----+----+------+------------+-----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|total_salary|salary_rank|
+-----+------+---------+----+----------+----+----+------+------------+-----------+
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|        NULL|          1|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|        NULL|          1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|        NULL|          1|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|        NULL|          2|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|        NULL|          3|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|NULL|    30|        NULL|          4|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|        NULL|          5|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|        NULL|          1|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|        NULL|          2|
| 77

In [110]:
#부서별 순위
from pyspark.sql.functions import rank, dense_rank, row_number, col, desc
from pyspark.sql.window import Window


# 부서별로 나누고 급여 기준으로 내림차순 정렬
dept_window = Window.partitionBy("deptno").orderBy(col("sal").desc())

# rank 함수 적용
dept_rank_df = emp_df.withColumn("salary_rank", rank().over(dept_window))

# 결과 확인
dept_rank_df.select("deptno", "ename", "sal", "salary_rank").show()


+------+------+----+-----------+
|deptno| ename| sal|salary_rank|
+------+------+----+-----------+
|    10|  KING|5000|          1|
|    10| CLARK|2450|          2|
|    10|MILLER|1300|          3|
|    20| SCOTT|3000|          1|
|    20|  FORD|3000|          1|
|    20| JONES|2975|          3|
|    20| ADAMS|1100|          4|
|    20| SMITH| 800|          5|
|    30| BLAKE|2850|          1|
|    30| ALLEN|1600|          2|
|    30|TURNER|1500|          3|
|    30|  WARD|1250|          4|
|    30|MARTIN|1250|          4|
|    30| JAMES| 950|          6|
|    70|  JACK|3200|          1|
+------+------+----+-----------+



In [111]:
#총누적 급여 sum('sal').over()


# 윈도우 정의: 정렬 기준만 있고 partition 없음 → 전체 누적합
window_all = Window.orderBy("sal")

# 누적합 컬럼 추가
emp_df.withColumn("cumulative_salary", sum("sal").over(window_all)).show()

+-----+------+---------+----+----------+----+----+------+------------+-----------------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|total_salary|cumulative_salary|
+-----+------+---------+----+----------+----+----+------+------------+-----------------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|        NULL|              800|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|NULL|    30|        NULL|             1750|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|        NULL|             2850|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|        1750|             5350|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|        2650|             5350|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|        NULL|             6650|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|        1500|             8150|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|        1900|             9750|
| 7782| CLARK|  MANAG

In [131]:
#부서별 누적 급여
dept_window = Window.partitionBy("deptno").orderBy("sal")

# 누적 급여 컬럼 추가
emp_df.withColumn("dept_cumulative_salary", sum("sal").over(dept_window)).show()


+-----+------+---------+----+----------+----+----+------+------------+----------------------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|total_salary|dept_cumulative_salary|
+-----+------+---------+----+----------+----+----+------+------------+----------------------+
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|        NULL|                  1300|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|        NULL|                  3750|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|        NULL|                  8750|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|        NULL|                   800|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|        NULL|                  1900|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|        NULL|                  4875|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|        NULL|                 10875|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|   

In [132]:
#부서별 평균급여와 직원 개별 급여 비교

from pyspark.sql.functions import when, round

dept_window = Window.partitionBy("deptno")

# 평균 급여 계산
emp_with_avg = emp_df.withColumn(
    "dept_avg_sal", 
    round(avg("sal").over(dept_window), 1)
)

# 비교: 개별 급여 vs 부서 평균
result_df = emp_with_avg.withColumn(
    "salary_vs_dept_avg",
    when(col("sal") > col("dept_avg_sal"), "Above Avg")
    .when(col("sal") < col("dept_avg_sal"), "Below Avg")
    .otherwise("Equal")
)

result_df.select("deptno", "ename", "sal", "dept_avg_sal", "salary_vs_dept_avg").show()

+------+------+----+------------+------------------+
|deptno| ename| sal|dept_avg_sal|salary_vs_dept_avg|
+------+------+----+------------+------------------+
|    10| CLARK|2450|      2916.7|         Below Avg|
|    10|  KING|5000|      2916.7|         Above Avg|
|    10|MILLER|1300|      2916.7|         Below Avg|
|    20| SMITH| 800|      2175.0|         Below Avg|
|    20| JONES|2975|      2175.0|         Above Avg|
|    20| SCOTT|3000|      2175.0|         Above Avg|
|    20| ADAMS|1100|      2175.0|         Below Avg|
|    20|  FORD|3000|      2175.0|         Above Avg|
|    30| ALLEN|1600|      1566.7|         Above Avg|
|    30|  WARD|1250|      1566.7|         Below Avg|
|    30|MARTIN|1250|      1566.7|         Below Avg|
|    30| BLAKE|2850|      1566.7|         Above Avg|
|    30|TURNER|1500|      1566.7|         Below Avg|
|    30| JAMES| 950|      1566.7|         Below Avg|
|    70|  JACK|3200|      3200.0|             Equal|
+------+------+----+------------+-------------

In [133]:
# 부서별 직업별 소계

emp_df.groupBy("deptno", "job") \
    .agg(sum("sal").alias("subtotal_salary")) \
    .orderBy("deptno", "job") \
    .show()

+------+---------+---------------+
|deptno|      job|subtotal_salary|
+------+---------+---------------+
|    10|    CLERK|           1300|
|    10|  MANAGER|           2450|
|    10|PRESIDENT|           5000|
|    20|  ANALYST|           6000|
|    20|    CLERK|           1900|
|    20|  MANAGER|           2975|
|    30|    CLERK|            950|
|    30|  MANAGER|           2850|
|    30| SALESMAN|           5600|
|    70|    CLERK|           3200|
+------+---------+---------------+



In [138]:
emp_df.cube('deptno', 'job').agg(count('*'), sum('sal'))\
    .orderBy('deptno', 'job').show() #평균급, 최대급, 최소급 컬럼 추가 비교

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|  NULL|     NULL|      15|   32225|
|  NULL|  ANALYST|       2|    6000|
|  NULL|    CLERK|       5|    7350|
|  NULL|  MANAGER|       3|    8275|
|  NULL|PRESIDENT|       1|    5000|
|  NULL| SALESMAN|       4|    5600|
|    10|     NULL|       3|    8750|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|     NULL|       5|   10875|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|     NULL|       6|    9400|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|     NULL|       1|    3200|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [134]:
dept_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("learning_spark_data/dept.csv")


In [135]:

dept_df.printSchema()

root
 |-- deptno: integer (nullable = true)
 |-- dname: string (nullable = true)
 |-- loc: string (nullable = true)



In [136]:
dept_df.dtypes


[('deptno', 'int'), ('dname', 'string'), ('loc', 'string')]

In [140]:
# 직원, 부서 조인1
emp_df.join(dept_df, on="deptno", how="inner").show()

+------+-----+------+---------+----+----------+----+----+------------+----------+--------+
|deptno|empno| ename|      job| mgr|  hiredate| sal|comm|total_salary|     dname|     loc|
+------+-----+------+---------+----+----------+----+----+------------+----------+--------+
|    20| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|        NULL|  RESEARCH|  DALLAS|
|    30| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|        1900|     SALES| CHICAGO|
|    30| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|        1750|     SALES| CHICAGO|
|    20| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|        NULL|  RESEARCH|  DALLAS|
|    30| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|        2650|     SALES| CHICAGO|
|    30| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|        NULL|     SALES| CHICAGO|
|    10| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|        NULL|ACCOUNTING|NEW YORK|
|    20| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|        NULL|  RESEARCH|  DALLAS|

In [141]:
# 직원, 부서 조인2
emp_dept_df = emp_df.join(dept_df, emp_df['deptno']==dept_df['deptno'])
emp_dept_df.show()

+-----+------+---------+----+----------+----+----+------+------------+------+----------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|total_salary|deptno|     dname|     loc|
+-----+------+---------+----+----------+----+----+------+------------+------+----------+--------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|        NULL|    20|  RESEARCH|  DALLAS|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|        1900|    30|     SALES| CHICAGO|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|        1750|    30|     SALES| CHICAGO|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|        NULL|    20|  RESEARCH|  DALLAS|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|        2650|    30|     SALES| CHICAGO|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|        NULL|    30|     SALES| CHICAGO|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|        NULL|    10|ACCOUNTING|NEW YORK|
| 7788| SCOTT|  ANAL

In [142]:
#필요한 컬럼만
join_df = emp_df.join(dept_df, on='deptno', how='inner')
join_df.select('ename', 'deptno', 'dname').show()

+------+------+----------+
| ename|deptno|     dname|
+------+------+----------+
| SMITH|    20|  RESEARCH|
| ALLEN|    30|     SALES|
|  WARD|    30|     SALES|
| JONES|    20|  RESEARCH|
|MARTIN|    30|     SALES|
| BLAKE|    30|     SALES|
| CLARK|    10|ACCOUNTING|
| SCOTT|    20|  RESEARCH|
|  KING|    10|ACCOUNTING|
|TURNER|    30|     SALES|
| ADAMS|    20|  RESEARCH|
| JAMES|    30|     SALES|
|  FORD|    20|  RESEARCH|
|MILLER|    10|ACCOUNTING|
+------+------+----------+



In [143]:
spark.stop()